In [147]:
import numpy as np
import pandas as pd
from PIL import Image

import torch
from torch import nn, optim
from torchvision import datasets, transforms

from sklearn.model_selection import train_test_split

from datasets import load_dataset

import matplotlib.pyplot as plt
from qbstyles import mpl_style

mpl_style(dark=True)

In [4]:
dataset = load_dataset("mnist", split="train[:100]")

README.md: 0.00B [00:00, ?B/s]

e:\HW files\Learner-s-Garden\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\daith\.cache\huggingface\hub\datasets--mnist. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [50]:
data_lst = [
    (transforms.ToTensor()(sample['image']) / 255.0, torch.tensor(sample['label']))
    for sample in dataset
]

In [38]:
data_lst[0][1]

tensor(5)

In [32]:
class SoftmaxRegression(nn.Module):
    def __init__(self, in_features, out_features):
        super(SoftmaxRegression, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
    
    def forward(self, x):
        return self.linear(x)
    
    def predict(self, x):
        self.eval()
        with torch.no_grad():
            return self.forward(x)

In [51]:
model = SoftmaxRegression(28 * 28, 10)
criterion = nn.CrossEntropyLoss()

In [113]:
optimizer = optim.SGD(model.parameters(), lr=3)
epochs = 100
losses = []

for epoch in range(1, epochs + 1):
    dataset_loss = 0
    model.train()
    for x, y in data_lst:
        yhat = model.forward(x.reshape(-1, 28 * 28))
        loss = criterion(yhat, y.reshape(1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        dataset_loss += loss.item()
    losses.append(dataset_loss / len(dataset))
    print(f"Epoch {epoch}/{epochs}: loss = {losses[-1]:.4f}")

Epoch 1/100: loss = 0.1876
Epoch 2/100: loss = 0.1875
Epoch 3/100: loss = 0.1874
Epoch 4/100: loss = 0.1872
Epoch 5/100: loss = 0.1871
Epoch 6/100: loss = 0.1870
Epoch 7/100: loss = 0.1869
Epoch 8/100: loss = 0.1867
Epoch 9/100: loss = 0.1866
Epoch 10/100: loss = 0.1865
Epoch 11/100: loss = 0.1864
Epoch 12/100: loss = 0.1862
Epoch 13/100: loss = 0.1861
Epoch 14/100: loss = 0.1860
Epoch 15/100: loss = 0.1859
Epoch 16/100: loss = 0.1857
Epoch 17/100: loss = 0.1856
Epoch 18/100: loss = 0.1855
Epoch 19/100: loss = 0.1854
Epoch 20/100: loss = 0.1852
Epoch 21/100: loss = 0.1851
Epoch 22/100: loss = 0.1850
Epoch 23/100: loss = 0.1849
Epoch 24/100: loss = 0.1847
Epoch 25/100: loss = 0.1846
Epoch 26/100: loss = 0.1845
Epoch 27/100: loss = 0.1844
Epoch 28/100: loss = 0.1843
Epoch 29/100: loss = 0.1841
Epoch 30/100: loss = 0.1840
Epoch 31/100: loss = 0.1839
Epoch 32/100: loss = 0.1838
Epoch 33/100: loss = 0.1836
Epoch 34/100: loss = 0.1835
Epoch 35/100: loss = 0.1834
Epoch 36/100: loss = 0.1833
E

In [115]:
model.predict(data_lst[1][0].reshape(-1, 28 * 28)).argmax()

tensor(0)

In [116]:
data_lst[1][1]

tensor(0)

In [117]:
parameters = list(model.parameters())

In [149]:
weights, biases = parameters

Image.fromarray(weights[9].reshape(28, 28).detach().numpy()).show()